In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from noisy_graphs import *

In [4]:
G = nx.read_edgelist('./graphs/collaboration_astrophysics')
A = nx.adjacency_matrix(G).todense()
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 18772
Number of edges: 198110


In [ ]:
Gprime, B = copy_graph(G)

# now run your algorithm on (G, Gprime) or on (A, B) !